# Import dependencies

In [1]:
# Data Manipulations and Preprocessing
import numpy as np
import pandas as pd
from sklearn.preprocessing import LabelEncoder
from sklearn.utils.class_weight import compute_class_weight
from imblearn.under_sampling import RandomUnderSampler

# Data Visualization
import seaborn as sns
import matplotlib.pyplot as plt

# Loading Classifiers
from xgboost import XGBClassifier
from sklearn.linear_model import LogisticRegression
from sklearn import neighbors
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier

# Model Evaluation Libraries
from sklearn.metrics import accuracy_score

# Tuning & Splitting Libraries
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB

# Services
import services.ModelStageService as sds

# utils
import datetime
import utils.EDAUtils as eda_utils

sns.set_style('darkgrid')
sns.color_palette('pastel')

stage_data_io_service = sds.ModelStageService(previous_stage_name=sds.PREPROCESSING_STAGE, current_stage_name=sds.FEATURE_ENGINEERING_STAGE)

[(0.6313725490196078, 0.788235294117647, 0.9568627450980393),
 (1.0, 0.7058823529411765, 0.5098039215686274),
 (0.5529411764705883, 0.8980392156862745, 0.6313725490196078),
 (1.0, 0.6235294117647059, 0.6078431372549019),
 (0.8156862745098039, 0.7333333333333333, 1.0),
 (0.8705882352941177, 0.7333333333333333, 0.6078431372549019),
 (0.9803921568627451, 0.6901960784313725, 0.8941176470588236),
 (0.8117647058823529, 0.8117647058823529, 0.8117647058823529),
 (1.0, 0.996078431372549, 0.6392156862745098),
 (0.7254901960784313, 0.9490196078431372, 0.9411764705882353)]

# Load data

In [ ]:
# Reading text from txt file
df = stage_data_io_service.run_or_load_stage_data(reload_stage=False)

# Feature Engineering and Data Processing for Model Building

## Adding Synthetic Replenishment Data for Accurate Calculation of Credit Debt Repayment Stability (CDRS) Ratio

Before calculating the Credit Debt Repayment Stability (CDRS) ratio, it is crucial to account for all transactions affecting the available balance, especially replenishments. However, the dataset lacks direct information on replenishment transactions, except for specific cases like reversal transactions. This absence introduces potential inaccuracies when calculating repayment stability.

To resolve this issue without altering the data schema by adding new markers, we will synthetically generate missing replenishment transactions. The approach works as follows:

1. **Data Grouping and Sorting**: We group the transactions by account number and arrange them in chronological order to establish a clear transaction flow.
   
2. **Pairwise Transaction Comparison**: After sorting, we compare each transaction with the preceding one. If the available balance of the current transaction exceeds that of the previous one and the previous transaction is not classified as a reversal or address verification, we infer that a replenishment transaction was missing between the two.

3. **Synthetic Replenishment Insertion**: For each identified gap, a synthetic replenishment transaction is inserted with a calculated amount and timestamp. This ensures that all changes in the available balance are accurately reflected in the dataset, allowing for a more precise calculation of the CDRS ratio.

By synthesizing replenishment data in this way, we can achieve a more accurate and realistic assessment of credit debt 
repayment behavior without distorting the original dataset's structure.


In [ ]:
def add_skipped_replenishment_transactions(df):
    """
    Adds synthetic replenishment transactions to a DataFrame of financial transactions.

    The method processes transaction data for each account, ensuring replenishment transactions 
    are inserted when missing. It works by grouping transactions by account number, 
    sorting them chronologically, and checking if a replenishment transaction is needed 
    between consecutive transactions. If the available balance of a later transaction 
    is greater than the previous one (and the previous transaction is not a reversal or address verification), 
    a replenishment transaction is added synthetically.

    Parameters:
    df (pd.DataFrame): DataFrame containing transaction data.

    Returns:
    pd.DataFrame: The original DataFrame with added synthetic replenishment transactions. 
    The output will be sorted by 'accountNumber' and 'transactionDateTime'.
    """

    # Sort data by account number and transaction datetime
    df = df.sort_values(['accountNumber', 'transactionDateTime']).reset_index(drop=True)

    # List to store new replenishment transactions
    replenishments = []

    # Group transactions by account number
    grouped = df.groupby('accountNumber')

    # Iterate over each account group
    for account, group in grouped:
        # Create a replenishment before the first transaction
        first_transaction = group.iloc[0]
        replenish_row = first_transaction.copy()
        replenish_row['transactionDateTime'] = first_transaction['transactionDateTime'] - datetime.timedelta(days=1)
        replenish_row['transactionAmount'] = first_transaction['creditLimit']
        replenish_row['enteredCVV'] = first_transaction['cardCVV']
        replenish_row['transactionType'] = 'REPLENISHMENT'
        replenish_row['isFraud'] = False
        replenishments.append(replenish_row)

        # Process the rest of the transactions
        for i in range(1, len(group)):
            current_transaction = group.iloc[i]
            previous_transaction = group.iloc[i - 1]

            # Check if a replenishment is needed between transactions
            if (current_transaction['availableMoney'] >= previous_transaction['availableMoney'] and
                    previous_transaction['transactionAmount'] != 0 and
                    previous_transaction['transactionType'] not in ['REVERSAL', 'ADDRESS_VERIFICATION']):
                # Calculate the replenishment datetime
                difference = current_transaction['transactionDateTime'] - previous_transaction['transactionDateTime']
                replenishment_transaction_date_time = (
                    current_transaction['transactionDateTime'] if difference <= datetime.timedelta(0)
                    else current_transaction['transactionDateTime'] - difference / 2
                )

                # Create replenishment transaction
                replenish_row = previous_transaction.copy()
                replenish_row['transactionDateTime'] = replenishment_transaction_date_time
                replenish_row['availableMoney'] = previous_transaction['availableMoney'] - previous_transaction[
                    'transactionAmount']
                replenish_row['creditDebt'] = previous_transaction['creditDebt'] + previous_transaction[
                    'transactionAmount']
                replenish_row['transactionAmount'] = current_transaction['availableMoney'] - replenish_row[
                    'availableMoney']
                replenish_row['enteredCVV'] = previous_transaction['cardCVV']
                replenish_row['transactionType'] = 'REPLENISHMENT'
                replenish_row['isFraud'] = False

                # Append replenishment to the list
                replenishments.append(replenish_row)

    # Convert replenishments to DataFrame and merge with the original DataFrame
    replenishment_df = pd.DataFrame(replenishments)
    result_df = pd.concat([df, replenishment_df]).sort_values(['accountNumber', 'transactionDateTime']).reset_index(
        drop=True)

    return result_df


# Example call of the function
df = stage_data_io_service.run_or_load_snapshot_data('add_skipped_replenishment_transactions', add_skipped_replenishment_transactions, df, False)


In [ ]:
# Check dimension after adding new rows

eda_utils.data_summary(df)

## Credit debt repayment stability ratio (CDRS Ratio)

### Overview
The primary goal of this analysis is to explore the relationship between a customer's spending and repayment patterns and the likelihood of fraudulent activity (`isFraud`). Specifically, we calculate a **stability coefficient** that reflects how consistently a customer closes their credit debt over time, then analyze how this coefficient correlates with fraud probability.

### Key Variables

1. **Spending Percentage**: The percentage of the credit limit that was spent between full debt repayments.
 $$ \text{Spent Percentage} = \left( \frac{\text{Total Amount Spent}}{\text{Credit Limit}} \right) \times 100 $$

2. **Days Between Resets**: The number of days between two consecutive events where the available balance equals the credit limit (i.e., the debt is fully repaid).

### Stability ratio

To measure customer stability, we consider two factors:
- The **average spending percentage** between resets.
- The **average number of days** between resets.

These two values are normalized using Min-Max normalization to bring them into the range of [0, 1]. The final stability coefficient is computed as:

$$ \text{CDRS Ratio} = 1 - (\text{Normalized Spending Percentage Between Resets} \times \text{Normalized Days Between Resets}) $$

This coefficient ranges from 0 to 1, where:
- A value closer to **1** indicates a more stable customer (less spending, more frequent repayments).
- A value closer to **0** indicates less stable repayment behavior.

### Hypothesis
We hypothesize that a higher stability coefficient (more stable customers) correlates with a lower probability of fraudulent activity. Conversely, lower stability may indicate higher fraud risk.

### Fraud Analysis
Once the CDRS Ratio is calculated for each account, we group the accounts into ranges (bins) of stability coefficients and calculate the percentage of fraudulent transactions (`isFraud = True`) in each range. The results are visualized to test the hypothesis.

### Conclusion
This analysis can help identify patterns between customer behavior and fraud risk, providing valuable insights for improving fraud detection models.

### Nice to fix and improve
- Investigate anomaly in cdrs-ratio range from 0.222-0.333 -- 0.333-0.444
- Improve scaling function (when size of input is 1)
- Investigate and fix when normalized `avg_norm_days_between_resets` is NaN (reason might be in scaling function)


In [ ]:

# Function for Min-Max normalization
def min_max_normalize(values):
    """
    Normalizes a list of values using Min-Max normalization.

    Parameters:
    values: list or array of numerical values to be normalized.

    Returns:
    Normalized numpy array of values in range [0, 1].
    """
    if len(values) > 1:
        return (np.array(values) - min(values)) / (max(values) - min(values))
    else:
        return np.array([1])


# Function to visualize fraud percentage based on stability coefficient
def visualize_fraud_vs_cdrs_ratio(fraud_stats):
    """
    Visualizes the fraud percentage based on cdrs ratio ranges.

    Parameters:
    fraud_stats: DataFrame containing fraud statistics grouped by cdrs ratio ranges.
    """
    plt.figure(figsize=(10, 6))
    plt.plot(fraud_stats.index.astype(str), fraud_stats['fraud_percentage'], marker='o', linestyle='-', color='r')
    plt.title('Fraud Percentage vs CDRS Ratio')
    plt.xlabel('Credit debt repayment stability ratio Range')
    plt.ylabel('Fraud Percentage')
    plt.xticks(rotation=45)
    plt.grid(True)
    plt.tight_layout()
    plt.show()


def calculate_account_stability_stats(account, transactions, reset_indices, stability_data):
    days_between_resets = []
    spent_percentages = []
    # Calculate the amount spent and days between credit limit resets
    for i in range(1, len(reset_indices)):
        start_idx = (reset_indices[i - 1]) + 1  # not include transaction of replenishment
        end_idx = (reset_indices[i]) - 1  # not include transaction of replenishment

        # Exclude replenishment transactions from the spending calculation
        transaction_slice = transactions.loc[start_idx:end_idx]
        non_replenishment_transactions = transaction_slice[
            (transaction_slice['transactionType'] != 'REPLENISHMENT') &
            (transaction_slice['transactionType'] != 'ADDRESS_VERIFICATION') &
            (transaction_slice['transactionType'] != 'REVERSAL')
            ]

        # Calculate the total amount spent between resets
        total_spent = non_replenishment_transactions['transactionAmount'].sum()
        credit_limit = transactions.loc[start_idx, 'creditLimit']

        spent_percentage = abs((total_spent / credit_limit) * 100)
        spent_percentages.append(spent_percentage)

        # Calculate the number of days between the two resets
        start_date = transactions.loc[start_idx, 'transactionDateTime']
        end_date = transactions.loc[end_idx, 'transactionDateTime']
        days_between = (end_date - start_date).days
        days_between_resets.append(days_between)

    # Normalize spent percentages and days between resets
    norm_spent_percentages = min_max_normalize(spent_percentages)
    norm_days_between_resets = min_max_normalize(days_between_resets)
    # Calculate the average normalized values
    avg_norm_spent_percentage = np.mean(norm_spent_percentages)
    avg_norm_days_between_resets = np.mean(norm_days_between_resets)
    # Stability ratio: higher values mean the customer is more stable
    cdrs_ratio = 1 - (avg_norm_spent_percentage * avg_norm_days_between_resets)
    stability_data.append({
        'accountNumber': account,
        'creditLimit': transactions.iloc[0].creditLimit,
        'avg_norm_spent_percentage': avg_norm_spent_percentage,
        'avg_norm_days_between_resets': avg_norm_days_between_resets,
        'cdrs_ratio': cdrs_ratio
    })

def calculate_fraud_stats(df, stability_stats):
    # Merge with the original DataFrame on accountNumber
    df_merged = df[['accountNumber', 'isFraud']].drop_duplicates().merge(stability_stats, on='accountNumber',
                                                                         how='left')
    # Create bins for stability coefficient ranges
    df_merged['cdrs_range'] = pd.cut(df_merged['cdrs_ratio'], bins=np.linspace(0, 1, 10))
    # Calculate fraud percentage in each stability coefficient range
    fraud_stats = df_merged.groupby('cdrs_range').agg(
        fraud_count=('isFraud', lambda x: (x == True).sum()),
        total_count=('isFraud', 'count')
    )
    fraud_stats['fraud_percentage'] = (fraud_stats['fraud_count'] / fraud_stats['total_count']) * 100
    return fraud_stats

def calculate_cdrs_ratio(df, credit_cover_threshold):
    """
    Calculates the credit debt repayment stability coefficient for each account and visualizes fraud probability based on the coefficient.

    Parameters:
    df: DataFrame containing transaction data for various accounts.
    credit_cover_threshold (ex. 0.2, 0.7, 1): float, the percentage threshold for considering replenishment transactions to be included in 
    stability calculations. Only replenishment transactions that result in an available balance exceeding 
    credit_cover_threshold% of the credit limit will be considered.

    Returns:
    Updated DataFrame with stability coefficient added, and fraud statistics DataFrame.
    """
    df = df.sort_values(['accountNumber', 'transactionDateTime'])

    stability_data = []

    # Loop through each account and calculate stability coefficient
    for account, transactions in df.groupby('accountNumber'):
        transactions = transactions.reset_index(drop=True)

        # Find indices where credit limit is reset (repaid based on the X percent threshold)
        reset_indices = transactions[
            (transactions['transactionType'] == 'REPLENISHMENT') &
            ((transactions['transactionAmount'] + transactions['availableMoney']) >= transactions[
                'creditLimit'] * credit_cover_threshold)
            ].index.tolist()

        if len(reset_indices) < 2:
            continue  # Skip accounts with insufficient data for calculation. Ratio for these account will be predicted by other model

        calculate_account_stability_stats(account, transactions, reset_indices, stability_data)

    # Create DataFrame with stability coefficients for each account
    stability_stats = pd.DataFrame(stability_data)

    fraud_stats = calculate_fraud_stats(df, stability_stats)

    # Add stability coefficient to the original DataFrame
    df = df.merge(stability_stats, on='accountNumber', how='left')

    return df, fraud_stats, stability_stats


# Call the function to analyze and visualize
df_with_stability, fraud_stats, tuned_stability_stats = calculate_cdrs_ratio(df, credit_cover_threshold=1)
visualize_fraud_vs_cdrs_ratio(fraud_stats)

In [ ]:
eda_utils.data_summary(df_with_stability)

## Predicting CDRS ratio for account with insufficient replenishments according to threshold


In [ ]:

df = df.sort_values(['accountNumber', 'transactionDateTime'])

stability_data = []

for account, transactions in df.groupby('accountNumber'):
    transactions = transactions.reset_index(drop=True)

    reset_indices = transactions[transactions['transactionType'] == 'REPLENISHMENT'].index.tolist()

    if len(reset_indices) < 2:
        reset_indices = [transactions.head(1).index.item(), transactions.tail(1).index.item()]

    calculate_account_stability_stats(account, transactions, reset_indices, stability_data)

stability_stats_full = pd.DataFrame(stability_data)
stability_stats_full['cdrs_ratio'] = np.nan

merged_df = stability_stats_full[['accountNumber']].merge(
    tuned_stability_stats[['accountNumber', 'cdrs_ratio']],
    on='accountNumber',
    how='left'
)

stability_stats_full['cdrs_ratio'] = merged_df['cdrs_ratio']

stability_stats_full['cdrs_range'] = pd.cut(stability_stats_full['cdrs_ratio'], bins=np.linspace(0, 1, 10))
stability_stats_full


In [ ]:
stability_stats_full.describe()

In [ ]:
eda_utils.data_summary(stability_stats_full)

In [ ]:
sns.heatmap(stability_stats_full[
                ['creditLimit', 'cdrs_ratio', 'avg_norm_days_between_resets', 'avg_norm_spent_percentage']].corr(),
            cmap="Reds", annot=True)

In [ ]:
both_avg_stats_is_na_accounts = stability_stats_full[(stability_stats_full['avg_norm_days_between_resets'].isnull()) & (
    stability_stats_full['avg_norm_spent_percentage'].isnull())]['accountNumber']
stab_df_both_avg_stats_is_na_accounts = stability_stats_full[
    stability_stats_full['accountNumber'].isin(both_avg_stats_is_na_accounts)]
stab_df_both_avg_stats_is_na_accounts

In [ ]:
gen_df_with_both_invalid_avg_stats = df[df['accountNumber'].isin(both_avg_stats_is_na_accounts)]
print(
    f'Count of account transactions where both necessary params avg (spent percentage, avg days btw resets) is nan in stability stats: {gen_df_with_both_invalid_avg_stats.shape[0]}')
eda_utils.calculate_percentage(gen_df_with_both_invalid_avg_stats.shape[0], df.shape[0])
print(
    f'Count of frauds transactions in gen dataset by these account numbers: {gen_df_with_both_invalid_avg_stats[gen_df_with_both_invalid_avg_stats["isFraud"] == True].shape[0]}')

In [ ]:
df.drop(index=gen_df_with_both_invalid_avg_stats.index, inplace=True)
stability_stats_full.drop(index=stab_df_both_avg_stats_is_na_accounts.index, inplace=True)
eda_utils.data_summary(stability_stats_full)

In [ ]:
stab_df_invalid_spent_avg_stats_is_na_accounts = \
    stability_stats_full[stability_stats_full['avg_norm_spent_percentage'].isnull()]['accountNumber']
gen_df_with_invalid_spent_avg_stats = df[df['accountNumber'].isin(stab_df_invalid_spent_avg_stats_is_na_accounts)]

print(
    f'Count of account transactions where avg spent percentage btw resets in nan in stability stats: {gen_df_with_invalid_spent_avg_stats.shape[0]}')
eda_utils.calculate_percentage(gen_df_with_invalid_spent_avg_stats.shape[0], df.shape[0])
print(
    f'Count of frauds transactions in gen dataset by these account numbers: {gen_df_with_invalid_spent_avg_stats[gen_df_with_invalid_spent_avg_stats["isFraud"] == True].shape[0]}')

In [ ]:
df[df['accountNumber'] == '259711806']

In [ ]:
df.drop(index=gen_df_with_invalid_spent_avg_stats.index, inplace=True)
stability_stats_full.drop(index=stab_df_invalid_spent_avg_stats_is_na_accounts.index, inplace=True)
eda_utils.data_summary(stability_stats_full)

In [ ]:

days_avg_stat_is_na_accounts = stability_stats_full[stability_stats_full['avg_norm_days_between_resets'].isnull()][
    'accountNumber']
stab_df_days_avg_stats_is_na_accounts = stability_stats_full[
    stability_stats_full['accountNumber'].isin(days_avg_stat_is_na_accounts)]
stab_df_days_avg_stats_is_na_accounts

In [ ]:
gen_df_with_invalid_days_avg_stats = df[df['accountNumber'].isin(days_avg_stat_is_na_accounts)]
print(
    f'Count of account transactions where avg days btw resets in nan in stability stats: {gen_df_with_invalid_days_avg_stats.shape[0]}')
eda_utils.calculate_percentage(gen_df_with_invalid_days_avg_stats.shape[0], df.shape[0])
print(
    f'Count of frauds transactions in gen dataset by these account numbers: {gen_df_with_invalid_days_avg_stats[gen_df_with_invalid_days_avg_stats["isFraud"] == True].shape[0]}')

eda_utils.calculate_percentage(gen_df_with_invalid_days_avg_stats[gen_df_with_invalid_days_avg_stats["isFraud"] == True].shape[0],
                     df[df['isFraud'] == True].shape[0])

In [ ]:
df[df['accountNumber'] == '155977598']

In [ ]:
df.drop(index=gen_df_with_invalid_days_avg_stats.index, inplace=True)
stability_stats_full.drop(index=stab_df_days_avg_stats_is_na_accounts.index, inplace=True)
eda_utils.data_summary(stability_stats_full)

In [ ]:
cdrs_ratio_is_0_accounts = stability_stats_full[stability_stats_full['cdrs_ratio'] == 0]['accountNumber']
stab_df_cdrs_ratio_is_0 = stability_stats_full[stability_stats_full['accountNumber'].isin(cdrs_ratio_is_0_accounts)]
stab_df_cdrs_ratio_is_0

In [ ]:
gen_df_cdrs_ratio_is_0 = df[df['accountNumber'].isin(cdrs_ratio_is_0_accounts)]
print(f'Count of account transactions where cdrs ratio is 0 in stability stats: {gen_df_cdrs_ratio_is_0.shape[0]}')
eda_utils.calculate_percentage(gen_df_cdrs_ratio_is_0.shape[0], df.shape[0])
print(
    f'Count of frauds transactions in gen dataset by these account numbers: {gen_df_cdrs_ratio_is_0[gen_df_cdrs_ratio_is_0["isFraud"] == True].shape[0]}')

eda_utils.calculate_percentage(gen_df_cdrs_ratio_is_0[gen_df_cdrs_ratio_is_0["isFraud"] == True].shape[0],
                     df[df['isFraud'] == True].shape[0])


In [ ]:
df.drop(index=gen_df_cdrs_ratio_is_0.index, inplace=True)
stability_stats_full.drop(index=cdrs_ratio_is_0_accounts.index, inplace=True)
eda_utils.data_summary(stability_stats_full)

In [ ]:
stability_stats_full_test_pred_df = stability_stats_full.copy()
stability_stats_full_test_pred_df.dropna(inplace=True)
cdrs_range_encoder = LabelEncoder()

stability_stats_full_test_pred_df['cdrs_range_encoded'] = cdrs_range_encoder.fit_transform(
    stability_stats_full_test_pred_df['cdrs_range'])
eda_utils.data_summary(stability_stats_full_test_pred_df)


In [ ]:
X_stats_full = stability_stats_full_test_pred_df.drop(
    columns=['cdrs_range', 'cdrs_range_encoded', 'cdrs_ratio', 'accountNumber'])
Y_stats_full = stability_stats_full_test_pred_df['cdrs_range_encoded']

X_stats_full_train, X_stats_full_test, Y_stats_full_train, Y_stats_full_test = train_test_split(X_stats_full,
                                                                                                Y_stats_full,
                                                                                                test_size=0.2,
                                                                                                random_state=101)

In [ ]:
cdrs_range_encoded_weights = compute_class_weight('balanced', classes=Y_stats_full_train.unique(), y=Y_stats_full_train)
print(f'Cdrs range distribution weights in train dataset {cdrs_range_encoded_weights}')

In [ ]:


# Define the models and their hyperparameters
models_dict = {
    "LogisticRegression": {
        "model": LogisticRegression(random_state=101, multi_class='auto'),
        "params": {
            "solver": ['newton-cg', 'lbfgs', 'liblinear'],
            "C": [0.1, 1, 10]
        }
    },
    "GaussianNB": {
        "model": GaussianNB(),
        "params": {
            "var_smoothing": [1e-09, 1e-08, 1e-07, 1e-06]
        }
    },
    "KNeighborsClassifier": {
        "model": neighbors.KNeighborsClassifier(),
        "params": {
            "n_neighbors": [3, 5, 7, 9],
            "weights": ['uniform', 'distance'],
            "metric": ['euclidean', 'manhattan']
        }
    },
    "RandomForest": {
        "model": RandomForestClassifier(random_state=101),
        "params": {
            'n_estimators': [50, 100, 200],
            'max_depth': [None, 10, 20, 30],
            'min_samples_split': [2, 5, 10]
        }
    },
    "DecisionTreeClassifier": {
        "model": DecisionTreeClassifier(),
        "params": {
            "criterion": ["gini", "entropy"],
            "max_depth": list(range(2,25,1)), 
            "min_samples_leaf": list(range(1,20,1))
        }
    }
}



In [ ]:
from sklearn.metrics import accuracy_score, roc_curve, auc, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
import joblib

def tune_and_evaluate_models(models, X_train, X_test, Y_train, Y_test):
    """
    Tunes each model in the provided dictionary, selects the best model based on accuracy, 
    and returns the best model with its name and accuracy.
    
    Args:
        models (dict): Dictionary where keys are model names and values are dictionaries
                       with keys "model" and "params" for each model.
        X_train, X_test: Training and testing features.
        Y_train, Y_test: Training and testing labels.

    Returns:
        best_model: The model with the highest accuracy after tuning.
        best_model_name (str): Name of the best model.
        best_accuracy (float): Best model's accuracy on the test set.
    """
    best_model = None
    best_accuracy = 0
    best_model_name = None

    for model_name, config in models.items():
        grid_search = GridSearchCV(config["model"], config["params"], cv=5, scoring='accuracy')
        grid_search.fit(X_train, Y_train)
        
        best_estimator = grid_search.best_estimator_
        pred = best_estimator.predict(X_test)
        accuracy = accuracy_score(Y_test, pred)

        print(f'Best parameters for {model_name}: {grid_search.best_params_}')
        print(f'Accuracy for {model_name}: {accuracy * 100:.2f}%')

        if accuracy > best_accuracy:
            best_accuracy = accuracy
            best_model = best_estimator
            best_model_name = model_name

    print(f'\nBest model is {best_model_name} with an accuracy of {best_accuracy * 100:.2f}%')
    return best_model, best_model_name, best_accuracy

def save_model(model, model_name):
    """
    Saves the best model to a file.
    
    Args:
        model: The model to save.
        model_name (str): The name of the model, used for the filename.
    """
    filename = f'{model_name}.pkl'
    joblib.dump(model, filename)
    print(f'The best model ({model_name}) has been saved as "{filename}".')

def plot_roc_curve_model(model, X_test, Y_test):
    """
    Plots the ROC curve for the model.

    Args:
        model: The model to plot the ROC curve for.
        X_test, Y_test: Testing data.
    """
    # Predict probabilities for the positive class
    y_pred_proba = model.predict_proba(X_test)[:, 1]

    # Compute ROC curve
    fpr, tpr, thresholds = roc_curve(Y_test, y_pred_proba)
    roc_auc = auc(fpr, tpr)

    # Plot the ROC curve
    plt.figure()
    plt.plot(fpr, tpr, color='darkorange', lw=2, label=f'ROC curve (area = {roc_auc:.2f})')
    plt.plot([0, 1], [0, 1], color='navy', lw=2, linestyle='--')
    plt.xlim([0.0, 1.0])
    plt.ylim([0.0, 1.05])
    plt.xlabel('False Positive Rate')
    plt.ylabel('True Positive Rate')
    plt.title('Receiver Operating Characteristic (ROC) Curve')
    plt.legend(loc='lower right')
    plt.show()

def plot_confusion_matrix_model(model, X_test, Y_test):
    """
    Plots the confusion matrix for the model.

    Args:
        model: The model to plot the confusion matrix for.
        X_test, Y_test: Testing data.
    """
    pred = model.predict(X_test)
    cm = confusion_matrix(Y_test, pred)
    sns.heatmap(cm, annot=True, fmt="d", cmap="Blues")
    plt.title("Confusion Matrix")
    plt.xlabel("Predicted")
    plt.ylabel("Actual")
    plt.show()

def plot_feature_importance(model, feature_names):
    """
    Plots feature importance for models that have a 'feature_importances_' attribute.

    Args:
        model: The model to plot feature importance for.
        feature_names: List of feature names for labeling.
    """
    if hasattr(model, 'feature_importances_'):
        importances = model.feature_importances_
        indices = importances.argsort()[::-1]
        
        plt.figure(figsize=(10, 6))
        plt.title("Feature Importances")
        plt.bar(range(len(indices)), importances[indices], align="center")
        plt.xticks(range(len(indices)), [feature_names[i] for i in indices], rotation=90)
        plt.show()
    else:
        print("The selected model does not support feature importance plotting.")




In [ ]:
# Example of how to use these functions
# Assume models_dict is defined with the models and parameters you want to tune
cdrs_range_best_model, cdrs_range_best_model_name, cdrs_range_model_best_accuracy = tune_and_evaluate_models(models_dict, X_stats_full_train, X_stats_full_test, Y_stats_full_train, Y_stats_full_test)

In [ ]:
plot_confusion_matrix_model(cdrs_range_best_model, X_stats_full_test, Y_stats_full_test)
plot_feature_importance(cdrs_range_best_model, X_stats_full_train.columns)

In [ ]:
eda_utils.data_summary(stability_stats_full)

In [ ]:
empty_cdrs_ratio_accounts = stability_stats_full[stability_stats_full['cdrs_range'].isnull()]
data_for_cdrs_range_prediction = empty_cdrs_ratio_accounts.drop(columns=['cdrs_ratio', 'cdrs_range', 'accountNumber'])
encoded_predicted_cdrs_range = cdrs_range_best_model.predict(data_for_cdrs_range_prediction)
empty_cdrs_ratio_accounts['cdrs_range'] = cdrs_range_encoder.inverse_transform(encoded_predicted_cdrs_range)

mean_cdrs_ratio_per_range = stability_stats_full[stability_stats_full['cdrs_range'].notna()].groupby('cdrs_range')['cdrs_ratio'].mean()
def fill_na_with_mean(row):
    if pd.isna(row['cdrs_ratio']):
        return mean_cdrs_ratio_per_range[row['cdrs_range']]
    else:
        return row['cdrs_ratio']

empty_cdrs_ratio_accounts['cdrs_ratio'] = empty_cdrs_ratio_accounts.apply(fill_na_with_mean, axis=1)

stability_stats_full.loc[
    stability_stats_full['accountNumber'].isin(empty_cdrs_ratio_accounts['accountNumber']),
    ['cdrs_range', 'cdrs_ratio']
] = empty_cdrs_ratio_accounts[['cdrs_range', 'cdrs_ratio']].values

stability_stats_full


In [ ]:
eda_utils.data_summary(stability_stats_full)

In [ ]:
visualize_fraud_vs_cdrs_ratio(calculate_fraud_stats(df, stability_stats_full))

#### Add cdrs ratio and range to general dataset and drop synthetic replenishment transactions

In [ ]:
cdrs_ratio_data = stability_stats_full[['accountNumber', 'cdrs_ratio', 'cdrs_range']]
df = df.merge(cdrs_ratio_data, on='accountNumber', how='left')
df = df[df['transactionType'] != 'REPLENISHMENT']
df.head()

In [ ]:
eda_utils.data_summary(df)

### Creating a new feature (MATCH ENTERED CVV OR NOT)

In [ ]:
df['cvv_matched_status'] = [True if actual_cvv == entered_cvv else False for actual_cvv, entered_cvv in zip(df['cardCVV'], df['enteredCVV'])]
df.drop(columns=['cardCVV', 'enteredCVV'], inplace=True)
eda_utils.data_summary(df)

### Creating a new feature (Address change count)

In [ ]:
def add_address_change_count_column(df):
    """
    Adds a new column 'addressChangeCount' to the DataFrame with the number of address changes
    per account up to the time of each transaction.

    Parameters:
    df (pd.DataFrame): The dataset containing transactions.

    Returns:
    pd.DataFrame: Updated DataFrame with the new 'addressChangeCount' column.
    """
    # Convert columns with dates to datetime format
    df['dateOfLastAddressChange'] = pd.to_datetime(df['dateOfLastAddressChange'])
    df['transactionDateTime'] = pd.to_datetime(df['transactionDateTime'])

    # Sort by accountNumber and transactionDateTime to ensure correct order
    df = df.sort_values(by=['accountNumber', 'transactionDateTime'])

    # Initialize a new column for address change count
    df['addressChangeCount'] = 0

    # Group by account number to process each account's transactions individually
    for account, group in df.groupby('accountNumber'):
        address_changes = 0
        last_change_date = None

        # Iterate through each transaction for the account
        for idx, row in group.iterrows():
            # Check if the current transaction date is after the last address change
            if last_change_date is None or row['dateOfLastAddressChange'] > last_change_date:
                address_changes += 1
                last_change_date = row['dateOfLastAddressChange']

            # Set the count of address changes up to the current transaction
            df.at[idx, 'addressChangeCount'] = address_changes - 1

    return df

In [ ]:
df = add_address_change_count_column(df)

In [ ]:
eda_utils.data_summary(df)

In [ ]:
def find_suspicious_transactions(df, time_threshold_milliseconds):
    """
    Identify suspicious transactions within a specified time threshold.

    Parameters:
    df (pd.DataFrame): The transaction dataset, which includes 'accountNumber', 'transactionDateTime', and 'transactionAmount'.
    time_threshold_milliseconds (int): The time window in milliseconds within which multiple transactions of the same amount are flagged as suspicious.

    Returns:
    pd.DataFrame: The original DataFrame with an additional 'is_suspicious' column.
    """
    # Sort by 'accountNumber' and 'transactionDateTime' to arrange transactions chronologically by account
    df = df.sort_values(['accountNumber', 'transactionDateTime']).reset_index(drop=True)
    
    # Check for duplicate transaction amounts for each account using `transform` to maintain DataFrame shape
    is_duplicate_amount = df.groupby('accountNumber')['transactionAmount'].transform(lambda x: x.duplicated())
    
    # Calculate time difference between consecutive transactions for each account in milliseconds
    is_within_time_threshold = df.groupby('accountNumber')['transactionDateTime'].diff() <= pd.Timedelta(time_threshold_milliseconds, unit='milliseconds')
    
    # Combine conditions: duplicate amount and time difference within threshold
    df["is_suspicious"] = is_duplicate_amount & is_within_time_threshold
    
    return df

In [ ]:
df = find_suspicious_transactions(df, time_threshold_milliseconds=3 * 60 * 1000)
suspicious_df = df[df['is_suspicious'] == True]

print("Total number of suspicious transactions: {}".format(suspicious_df.shape[0]))
print("Total transaction amount of suspicious transactions: {}$".format(round(suspicious_df['transactionAmount'].sum(), 2)))
eda_utils.data_summary(df)

## Preparing df for Model building

In [ ]:
modelling_df = df.drop(columns=[
    'accountNumber',
    'transactionDateTime',
    'currentExpDate',
    'accountOpenDate',
    'dateOfLastAddressChange'
])
modelling_df.columns

In [ ]:
cat_cols = ['acqCountry', 'merchantCountryCode', 'posEntryMode', 'posConditionCode', 'merchantCategoryCode', 'transactionType', 'creditLimitRange', 'cdrs_range']

In [ ]:
label_encoder = LabelEncoder()

for cat in cat_cols:
    modelling_df[cat]= label_encoder.fit_transform(modelling_df[cat])

In [ ]:
modelling_df['cardPresent'] = [1 if cardPresent == True else 0 for cardPresent in modelling_df['cardPresent']]
modelling_df['expirationDateKeyInMatch'] = [1 if expirationDateKeyInMatch == True else 0 for expirationDateKeyInMatch in modelling_df['expirationDateKeyInMatch']]
modelling_df['isFraud'] = [1 if isFraud == True else 0 for isFraud in modelling_df['isFraud']]
modelling_df['cvv_matched_status'] = [1 if cvv_matched_status == True else 0 for cvv_matched_status in modelling_df['cvv_matched_status']]
modelling_df['is_suspicious'] = [1 if is_suspicious == True else 0 for is_suspicious in modelling_df['is_suspicious']]

In [ ]:
def show_fraud_distribution(df, target_column, title='Transaction Count by isFraud Value'):
    """
    Plot the distribution of transaction counts for fraud and non-fraud transactions.

    Parameters:
    df (pd.DataFrame): The DataFrame containing transaction data with a binary 'isFraud' column.

    Returns:
    None
    """
    print(f'Recorded transactions: {df.shape}')
    # Count the number of transactions where isFraud is True and False
    fraud_counts = target_column.value_counts()

    # Create a bar plot for the counts
    plt.figure(figsize=(8, 6))
    sns.barplot(hue=fraud_counts.index, y=fraud_counts.values, palette='viridis', legend=False)

    # Add title and labels
    plt.title(title)
    plt.xlabel('isFraud')
    plt.ylabel('Transaction Count')

    # Display values on top of each bar
    for i, v in enumerate(fraud_counts.values):
        plt.text(i, v + 100, str(v), ha='center', fontweight='bold')

    # Show the plot
    plt.show()


In [ ]:
show_fraud_distribution(modelling_df, modelling_df['isFraud'], 'Transaction Count by isFraud Value (without sampling)')

In [ ]:
target = modelling_df['isFraud']
features = modelling_df.drop('isFraud', axis=1)

# Performing Under Sampling to balance the dataset and minimize the computation time
sampled_modelling_df, sampled_target = RandomUnderSampler(sampling_strategy='majority',
                                                          random_state=3).fit_resample(features, target)

In [ ]:
show_fraud_distribution(sampled_modelling_df, sampled_target, 'Transaction Count by isFraud Value (after undersampling)')

In [ ]:
stage_data_io_service.write_stage_data(df)